# Extract Audio from Video File

In [3]:
from pathlib import Path
import shutil, subprocess

def have_ffmpeg() -> bool:
    return shutil.which("ffmpeg") is not None

def extract_audio_to_wav(
    input_video: str,
    output_wav: str = "meeting.wav",
    sr: int = 16000,
    mono: bool = True,
    normalize: bool = False,
    start: float | None = None,   
    end: float | None = None):
    
    if not have_ffmpeg():
        raise RuntimeError("ffmpeg not found on PATH. Install ffmpeg and retry.")

    in_path = Path(input_video)
    out_path = Path(output_wav)
    out_path.parent.mkdir(parents=True, exist_ok=True)

    cmd = ["ffmpeg", "-y", "-hide_banner", "-loglevel", "error"]

    if start is not None:
        cmd += ["-ss", str(float(start))]
    if end is not None:
        cmd += ["-to", str(float(end))]

    cmd += ["-i", str(in_path)]

    cmd += ["-vn"]  

    af = []
    if normalize:
        af.append("loudnorm=I=-16:LRA=11:TP=-1.5")

    if af:
        cmd += ["-af", ",".join(af)]

    if mono: cmd += ["-ac", "1"]
    cmd += ["-ar", str(int(sr))]

    cmd += ["-c:a", "pcm_s16le", str(out_path)]

    print("Running:", " ".join(cmd))
    subprocess.run(cmd, check=True)
    print(f"Wrote {out_path.resolve()}")
    return str(out_path)

In [4]:
extract_audio_to_wav("SEP_12_CLIF.mp4", "SEP_12_CLIF.wav")

Running: ffmpeg -y -hide_banner -loglevel error -i SEP_12_CLIF.mp4 -vn -ac 1 -ar 16000 -c:a pcm_s16le SEP_12_CLIF.wav


Output file #0 does not contain any stream


CalledProcessError: Command '['ffmpeg', '-y', '-hide_banner', '-loglevel', 'error', '-i', 'SEP_12_CLIF.mp4', '-vn', '-ac', '1', '-ar', '16000', '-c:a', 'pcm_s16le', 'SEP_12_CLIF.wav']' returned non-zero exit status 1.